In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
censuses_data = pd.read_csv("../../CSV/censuses.csv")
# censuses_data = censuses_data[censuses_data["census_year"] == 1900]
censuses_data.count()

line_number               4244
house_number              4090
street_name               4090
last_name                 4090
given_name                4090
relation                  4090
color_race                4244
sex                       4244
age_at_last_birthday      4244
occupation                4244
own_or_rent               4090
owned_free_or_mortgage    4090
house_or_farm             4090
my_notes                  4090
file_name                 4090
census_year               4244
grouped_occu              4244
dtype: int64

In [3]:
newdata = censuses_data.loc[:,["color_race","sex","age_at_last_birthday", "census_year", "grouped_occu"]]
newdata.count()

color_race              4244
sex                     4244
age_at_last_birthday    4244
census_year             4244
grouped_occu            4244
dtype: int64

In [4]:
newdata.groupby(['census_year']).grouped_occu.value_counts()

census_year  grouped_occu         
1900         Missing Occupations       321
             Unskilled                 277
             Student                   121
             Skilled Labor              84
             Skilled Professionals       5
1910         Unskilled                 116
             Skilled Labor              20
             Missing Occupations        14
             Skilled Professionals       4
1920         Unskilled                1580
             Missing Occupations      1068
             Skilled Labor             503
             Skilled Professionals     131
Name: grouped_occu, dtype: int64

In [5]:
def age_clean(data,under_1_value = 1, pandas = 'yes'):
    clean_age = []
    if pandas == 'yes':
        data_lst = data.tolist()
    else:
        data_lst = data
    for datum in data_lst:
        if 'month' in str(datum):
            if 'and' in str(datum):
                clean_age.append(str(datum).split(' ')[0])
            else:
                clean_age.append(str(under_1_value))
        else:
            clean_age.append(datum)
    return clean_age

In [6]:
# Cleaning up age
newdata.age_at_last_birthday = age_clean(newdata.age_at_last_birthday)

In [7]:
missing_occu_mask = (newdata['grouped_occu'] == "Missing Occupations")
df_missing_occu = newdata[missing_occu_mask]
df_missing_occu.count()

color_race              1403
sex                     1403
age_at_last_birthday    1403
census_year             1403
grouped_occu            1403
dtype: int64

In [8]:
# Working with the labelled occupations
df_modeldata = newdata[missing_occu_mask == False]
df_modeldata.count()

color_race              2841
sex                     2841
age_at_last_birthday    2841
census_year             2841
grouped_occu            2841
dtype: int64

In [9]:
X = df_modeldata.drop(["grouped_occu","census_year"], axis=1)
y = df_modeldata["grouped_occu"]

print(X.shape, y.shape)

(2841, 3) (2841,)


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=1/2)
print(X_train.shape, X_test.shape)

(1420, 3) (1421, 3)


In [11]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_ohe = ohe.fit_transform(X_train)
X_test_ohe = ohe.transform(X_test)

print(X_train_ohe.shape, X_test_ohe.shape)

(1420, 79) (1421, 79)


In [14]:
y_train_ohe = ohe.fit_transform(y_train)
y_test_ohe = ohe.transform(y_test)

print(y_train_ohe.shape, y_test_ohe.shape)

ValueError: Expected 2D array, got 1D array instead:
array=['Unskilled' 'Unskilled' 'Skilled Labor' ... 'Unskilled' 'Student'
 'Unskilled'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [13]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

classifier.fit(X_train_ohe,y_train)

print(f"Training Data Score: {classifier.score(X_train_ohe, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_ohe, y_test)}")

Training Data Score: 0.7147887323943662
Testing Data Score: 0.7065446868402533


D:\Users\kshah\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\kshah\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
